In [15]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# ------------------------------
# 1. Generar Caracteristicas_Equipos.csv
# ------------------------------

# Parámetros iniciales
num_equipos = 500
tipos_equipo = ['Motor', 'Generador', 'Compresor', 'Bomba']
fabricantes = ['Fabricante_A', 'Fabricante_B', 'Fabricante_C', 'Fabricante_D']
modelos = [f'Modelo_{i}' for i in range(1, 11)]

# Generar datos
data_caracteristicas = []
for equipo_id in range(1, num_equipos + 1):
    tipo_equipo = np.random.choice(tipos_equipo)
    fabricante = np.random.choice(fabricantes)
    modelo = np.random.choice(modelos)
    potencia_kw = np.random.randint(50, 500)  # Potencia entre 50 kW y 500 kW
    horas_recomendadas_revision = np.random.randint(500, 1000)  # Horas recomendadas entre 500 y 1000
    
    data_caracteristicas.append([equipo_id, tipo_equipo, fabricante, modelo, potencia_kw, horas_recomendadas_revision])

# Crear DataFrame
columns_caracteristicas = ['ID_Equipo', 'Tipo_Equipo', 'Fabricante', 'Modelo', 'Potencia_kW', 'Horas_Recomendadas_Revision']
df_caracteristicas = pd.DataFrame(data_caracteristicas, columns=columns_caracteristicas)

# Guardar como CSV
df_caracteristicas.to_csv('./out/Caracteristicas_Equipos.csv', index=False)
print("Dataset 'Caracteristicas_Equipos.csv' generado.")

# ------------------------------
# 2. Generar Registros_Condiciones.csv
# ------------------------------

# Parámetros iniciales
fecha_inicio = datetime(2021, 1, 1)
fecha_fin = datetime(2024, 12, 31)
dias_totales = (fecha_fin - fecha_inicio).days + 1
num_equipos = 500

# Generar datos
data_registros = []
reset_dates = {}  # Para almacenar fechas de reset por equipo

for equipo_id in range(1, num_equipos + 1):
    horas_operativas = np.random.randint(0, 721)  # Horas iniciales aleatorias entre 0 y 720
    reset_dates[equipo_id] = []  # Lista para almacenar fechas de reset
    
    for dia in range(dias_totales):
        fecha_actual = fecha_inicio + timedelta(days=dia)
        
        # Incrementar horas operativas en un número aleatorio entre 0 y 24
        incremento_horas = np.random.randint(0, 25)
        horas_operativas += incremento_horas
        
        # Aleatoriamente resetear las horas operativas (probabilidad del 5%)
        if np.random.rand() < 0.01:
            horas_operativas = np.random.randint(0, 24)  # Resetear a un valor aleatorio
            reset_dates[equipo_id].append(fecha_actual)  # Guardar la fecha de reset
        
        # Generar valores aleatorios para temperatura y vibración
        temperatura = np.random.uniform(20, 100)  # Temperatura entre 20°C y 100°C
        vibracion = np.random.uniform(0, 10)      # Vibración entre 0 mm/s y 10 mm/s
        
        # Añadir el registro a la lista de datos
        data_registros.append([
            len(data_registros) + 1,  # ID_Registro (único para cada fila)
            equipo_id,                # ID_Equipo
            fecha_actual.strftime('%Y-%m-%d'),  # Fecha
            temperatura,              # Temperatura_C
            vibracion,                # Vibracion_mm_s
            horas_operativas          # Horas_Operativas
        ])

# Crear DataFrame
columns_registros = ['ID_Registro', 'ID_Equipo', 'Fecha', 'Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']
df_registros = pd.DataFrame(data_registros, columns=columns_registros)

# Guardar como CSV
df_registros.sort_values(by=[ 'Fecha'], inplace=True)
df_registros=df_registros.reset_index(drop=True)  # Ordenar por fecha
df_registros['ID_Registro'] = df_registros.index + 1  # Reasignar ID_Registro

df_registros.to_csv('./out/Registros_Condiciones.csv', index=False)
print("Dataset 'Registros_Condiciones.csv' generado.")

# ------------------------------
# 3. Generar Historicos_Ordenes.csv
# ------------------------------

# Parámetros iniciales
tipos_mantenimiento = ['Preventivo', 'Correctivo']
ubicaciones = ['Ubicacion_A', 'Ubicacion_B', 'Ubicacion_C', 'Ubicacion_D']

# Generar datos
data_ordenes = []

for equipo_id in range(1, num_equipos + 1):
    ubicacion = np.random.choice(ubicaciones)  # Ubicación fija para el equipo
    horas_recomendadas_revision = df_caracteristicas.loc[df_caracteristicas['ID_Equipo'] == equipo_id, 'Horas_Recomendadas_Revision'].values[0]
    
    # Mantenimientos correctivos (basados en fechas de reset)
    for fecha_reset in reset_dates[equipo_id]:
        costo_correctivo = np.random.randint(500, 5000)  # Costo entre 500 y 5000
        duracion_correctivo = np.random.randint(1, 25)  # Duración entre 1 y 24 horas
        
        data_ordenes.append([
            len(data_ordenes) + 1,  # ID_Orden
            equipo_id,              # ID_Equipo
            fecha_reset.strftime('%Y-%m-%d'),  # Fecha
            'Correctivo',           # Tipo_Mantenimiento
            costo_correctivo,       # Costo_Mantenimiento
            duracion_correctivo,    # Duracion_Horas
            ubicacion               # Ubicacion
        ])
    
    # Mantenimientos preventivos (basados en horas recomendadas de revisión)
    fecha_actual = fecha_inicio
    while fecha_actual <= fecha_fin:
        # Calcular la próxima fecha de mantenimiento preventivo
        horas_preventivo = horas_recomendadas_revision * np.random.uniform(0.8, 1.2)  # ±20%
        dias_preventivo = int(horas_preventivo / 24)
        fecha_proximo_preventivo = fecha_actual + timedelta(days=dias_preventivo)
        
        if fecha_proximo_preventivo > fecha_fin:
            break
        
        costo_preventivo = np.random.randint(100, 1000)  # Costo entre 100 y 1000
        duracion_preventivo = np.random.randint(1, 3)    # Duración entre 1 y 2 horas
        
        data_ordenes.append([
            len(data_ordenes) + 1,  # ID_Orden
            equipo_id,              # ID_Equipo
            fecha_proximo_preventivo.strftime('%Y-%m-%d'),  # Fecha
            'Preventivo',           # Tipo_Mantenimiento
            costo_preventivo,       # Costo_Mantenimiento
            duracion_preventivo,    # Duracion_Horas
            ubicacion               # Ubicacion
        ])
        
        fecha_actual = fecha_proximo_preventivo

# Crear DataFrame
columns_ordenes = ['ID_Orden', 'ID_Equipo', 'Fecha', 'Tipo_Mantenimiento', 'Costo_Mantenimiento', 'Duracion_Horas', 'Ubicacion']
df_ordenes = pd.DataFrame(data_ordenes, columns=columns_ordenes)

# Guardar como CSV
df_ordenes.sort_values(by=[ 'Fecha'], inplace=True)
df_ordenes=df_ordenes.reset_index(drop=True)  # Ordenar por fecha
df_ordenes['ID_Orden'] = df_ordenes.index + 1  # Reasignar ID_Registro

df_ordenes.to_csv('./out/Historicos_Ordenes.csv', index=False)
print("Dataset 'Historicos_Ordenes.csv' generado.")

Dataset 'Caracteristicas_Equipos.csv' generado.
Dataset 'Registros_Condiciones.csv' generado.
Dataset 'Historicos_Ordenes.csv' generado.


## Analisis

In [16]:
from tabulate import tabulate

print('df_ordenes:',tabulate(df_ordenes.head(20), headers='keys', tablefmt='psql'))
print('df_registros:',tabulate((df_registros.head(20)), headers='keys', tablefmt='psql'))
print('df_caracteristicas:',tabulate((df_caracteristicas.head(5)), headers='keys', tablefmt='psql'))

df_ordenes: +----+------------+-------------+------------+----------------------+-----------------------+------------------+-------------+
|    |   ID_Orden |   ID_Equipo | Fecha      | Tipo_Mantenimiento   |   Costo_Mantenimiento |   Duracion_Horas | Ubicacion   |
|----+------------+-------------+------------+----------------------+-----------------------+------------------+-------------|
|  0 |          1 |         382 | 2021-01-01 | Correctivo           |                  4331 |               16 | Ubicacion_A |
|  1 |          2 |         350 | 2021-01-01 | Correctivo           |                  2927 |               12 | Ubicacion_A |
|  2 |          3 |         302 | 2021-01-02 | Correctivo           |                  3863 |                6 | Ubicacion_B |
|  3 |          4 |          55 | 2021-01-02 | Correctivo           |                  1556 |                2 | Ubicacion_D |
|  4 |          5 |          89 | 2021-01-02 | Correctivo           |                  3680 |      

In [17]:
from tabulate import tabulate

print('df_ordenes:',tabulate(df_ordenes.tail(20), headers='keys', tablefmt='psql'))
print('df_registros:',tabulate((df_registros.tail(20)), headers='keys', tablefmt='psql'))
print('df_caracteristicas:',tabulate((df_caracteristicas.tail(5)), headers='keys', tablefmt='psql'))

df_ordenes: +-------+------------+-------------+------------+----------------------+-----------------------+------------------+-------------+
|       |   ID_Orden |   ID_Equipo | Fecha      | Tipo_Mantenimiento   |   Costo_Mantenimiento |   Duracion_Horas | Ubicacion   |
|-------+------------+-------------+------------+----------------------+-----------------------+------------------+-------------|
| 31846 |      31847 |         277 | 2024-12-30 | Correctivo           |                  4596 |               20 | Ubicacion_B |
| 31847 |      31848 |          72 | 2024-12-31 | Preventivo           |                   394 |                1 | Ubicacion_D |
| 31848 |      31849 |          71 | 2024-12-31 | Preventivo           |                   991 |                2 | Ubicacion_C |
| 31849 |      31850 |         272 | 2024-12-31 | Preventivo           |                   136 |                2 | Ubicacion_B |
| 31850 |      31851 |         490 | 2024-12-31 | Preventivo           |      

In [18]:
print('df_ordenes:',df_ordenes.shape)
print('df_registros:',df_registros.shape)
print('df_caracteristicas:',df_caracteristicas.shape)

df_ordenes: (31866, 7)
df_registros: (730500, 6)
df_caracteristicas: (500, 6)


In [19]:
# resgitros por id_equipo, año y mes 

df_registros['Año'] = pd.to_datetime(df_registros['Fecha']).dt.year
df_registros['Mes'] = pd.to_datetime(df_registros['Fecha']).dt.month 
df_registros['Dia'] = pd.to_datetime(df_registros['Fecha']).dt.day 

df_registros.groupby(['ID_Equipo', 'Año', 'Mes']).count()

ID_Registro  Fecha  Temperatura_C  Vibracion_mm_s  \
ID_Equipo Año  Mes                                                      
1         2021 1             31     31             31              31   
               2             28     28             28              28   
               3             31     31             31              31   
               4             30     30             30              30   
               5             31     31             31              31   
...                         ...    ...            ...             ...   
500       2024 8             31     31             31              31   
               9             30     30             30              30   
               10            31     31             31              31   
               11            30     30             30              30   
               12            31     31             31              31   

                    Horas_Operativas  Dia  
ID_Equipo Año  Mes                         
1         2021 1                  31   31  
               2                  28   28  
               3                  31   31  
               4                  30   30  
               5                  31   31  
...                              ...  ...  
500       2024 8                  31   31  
               9                  30   30  
               10                 31   31  
               11                 30   30  
               12                 31   31  

[24000 rows x 6 columns]

In [20]:
# Ordenes por id_equipo, año y mes

df_ordenes['Año'] = pd.to_datetime(df_ordenes['Fecha']).dt.year
df_ordenes['Mes'] = pd.to_datetime(df_ordenes['Fecha']).dt.month
df_ordenes['Dia'] = pd.to_datetime(df_ordenes['Fecha']).dt.day

print('\n** Correctivo:')
print(df_ordenes[['ID_Equipo', 'Fecha', 'Tipo_Mantenimiento']
           ].loc[(df_ordenes['Tipo_Mantenimiento'] == 'Correctivo') & (df_ordenes['ID_Equipo'] == 1)].count())


print('\n** Preventivo:')
print(df_ordenes[['ID_Equipo', 'Fecha', 'Tipo_Mantenimiento']
           ].loc[(df_ordenes['Tipo_Mantenimiento'] == 'Preventivo') & (df_ordenes['ID_Equipo'] == 1)].count())


** Correctivo:
ID_Equipo             10
Fecha                 10
Tipo_Mantenimiento    10
dtype: int64

** Preventivo:
ID_Equipo             55
Fecha                 55
Tipo_Mantenimiento    55
dtype: int64
